#### Import Libraries

In [6]:
import openai
import pypdf
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
OPENAI_API_KEY = "API_KEY"

In [ ]:
from langchain.llms import OpenAI

llm = OpenAI(openai_api_key=OPENAI_API_KEY)

#### Using custom dataset

In [ ]:
pdf_reader = PyPDFLoader("/content/RAGPaper.pdf")
documents = pdf_reader.load()

#### RecursiveCharacterTextSplitter is a text splitter that splits the text into chunks.

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

# Create embedding
embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY)
db = FAISS.from_documents(documents=chunks, embedding=embeddings)

In [3]:
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate

In [ ]:
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate

CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template("""Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question

Chat History:
{chat_history}
Follow up input : {question}
standalone questions : """)

qa = ConversationalRetrievalChain.from_llm(llm=llm, retriever=db.as_retriever(),
                                           condense_question_prompt=CONDENSE_QUESTION_PROMPT,
                                           return_source_documents=True, verbose=False)

#### Ask a query

In [ ]:
chat_history = []
query = """What is this paper all about, explain in just 3-4 lines"""
result = qa.({"question":query, "chat_history":chat_history})
print(result["answer"])